https://www.lazard.com/research-insights/2023-levelized-cost-of-energyplus/

https://www.energy.gov/eere/solar/solar-photovoltaic-system-cost-benchmarks

battery capacity less than or equal to off peak requirement

stored power = peak_requirement - solar_capacity

off peak cost = off peak grid price \* (off peak requirement - solar capacity)

peak cost = peak requirement - solar capacity

minimize total cost

In [ ]:
%pip install pyomo

In [ ]:
import pyomo.environ as pyo
from solar_installations import INSTALLATION_SIZES

In [ ]:
model = pyo.ConcreteModel()


model.peak_load = pyo.Param(initialize=1)
model.off_peak_load = pyo.Param(initialize=5)
model.peak_grid_price = pyo.Param(initialize=0.5)
model.off_peak_grid_price = pyo.Param(initialize=0.5)
model.battery_cost_per_kw = pyo.Param(initialize=0.1)


model.SOLAR_SIZES = pyo.Set(initialize=INSTALLATION_SIZES.keys())
model.solar_size_flags = pyo.Var(model.SOLAR_SIZES, within=pyo.Binary)
model.solar_capacity = pyo.Var(within=pyo.NonNegativeReals)
model.solar_cost = pyo.Var(within=pyo.NonNegativeReals)

model.peak_grid_consumption = pyo.Var(within=pyo.NonNegativeReals)
model.off_peak_grid_usage = pyo.Var(within=pyo.NonNegativeReals)
model.battery_capacity = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, 4))

model.minimizeCost = pyo.Objective(
    expr=(model.peak_grid_price * model.peak_grid_consumption)
    + (model.off_peak_grid_price * model.off_peak_grid_usage)
    + model.solar_cost
    + (model.battery_cost_per_kw * model.battery_capacity),
    sense=pyo.minimize,
)

model.off_peak_constraint = pyo.Constraint(
    expr=model.off_peak_load <= model.off_peak_grid_usage + model.battery_capacity
)

model.peak_constraint = pyo.Constraint(
    expr=model.peak_load <= model.peak_grid_consumption + model.solar_capacity
)

model.battery_charging_constraint = pyo.Constraint(
    expr=model.battery_capacity <= model.solar_capacity - model.peak_load
)


def installed_solar_constraint(model):
    sum_term = sum(
        INSTALLATION_SIZES[i] * model.solar_size_flags[i] for i in model.SOLAR_SIZES
    )
    return model.solar_cost == sum_term


model.consumption_constraint = pyo.Constraint(rule=installed_solar_constraint)
model.sos1_constraint = pyo.SOSConstraint(var=model.solar_size_flags, sos=1)


def required_power(model):
    return model.solar_capacity >= 6


model.required_power_constraint = pyo.Constraint(rule=required_power)


def solar_capacity_constraint(model):
    return model.solar_capacity == sum(
        size * model.solar_size_flags[size] for size in model.SOLAR_SIZES
    )


model.solar_capacity_constraint = pyo.Constraint(rule=solar_capacity_constraint)

optimizer = pyo.SolverFactory("gurobi")
optimizer.solve(model)
print(
    f"solar capacity: {model.solar_capacity.value} kW, battery capacity: {model.battery_capacity.value} kWh, off peak grid usage: {model.off_peak_grid_usage.value} kW, peak grid consumption: {model.peak_grid_consumption.value} kW"
)
print(model.display())

In [ ]:
print(model.display())